### 라이브러리

In [33]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib 
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams['font.family'] = "AppleGothic"

from sklearn.ensemble import RandomForestRegressor

from pycaret.regression import *
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error

### 데이터 처리

In [34]:
train = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/train.csv')
test = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/test.csv')
submission = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/sample_submission.csv')

In [35]:
weekdayrankfordinner = {
    '월' : 1,
    '화' : 2,
    '수' : 4,
    '목' : 3,
    '금' : 5
} 

train['요일(석식)'] = train['요일'].map(weekdayrankfordinner)
test['요일(석식)'] = test['요일'].map(weekdayrankfordinner)

In [36]:
train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

train['년'] = train['일자'].dt.year
train['월'] = train['일자'].dt.month
train['일'] = train['일자'].dt.day
train['주'] = train['일자'].dt.isocalendar().week
train['요일'] = train['일자'].dt.weekday


test['년'] = test['일자'].dt.year
test['월'] = test['일자'].dt.month
test['일'] = test['일자'].dt.day
test['주'] = test['일자'].dt.isocalendar().week
test['요일'] = test['일자'].dt.weekday

train['주'] = train['주'].astype('int') # '주'의 타입이 Uint로 사용이 안되어 int로 변경해줌.
test['주'] = test['주'].astype('int')

In [37]:
drops = ['조식메뉴', '중식메뉴', '석식메뉴']

train = train.drop(drops, axis=1)
test = test.drop(drops, axis=1)

In [38]:
train['식사가능자수'] = (train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수'])
test['식사가능자수'] = (test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수'])

In [39]:
# month1 = {
#     1:3,
#     2:1,
#     3:2,
#     4:6,
#     5:7,
#     6:8,
#     7:10,
#     8:9,
#     9:5,
#     10:4,
#     11:11,
#     12:12
# }

month1 = {
    6:12,
	5:11,
	8:10,
	7:9	,
	10:8,
	11:7,
	9:6,
	4:5,
	2:4,
	3:3,
	1:2,
	12:1
}



month2 = {
    1:11,
    2:2,
    3:1,
    4:4,
    5:8,
    6:6,
    7:9,
    8:7,
    9:5,
    10:3,
    11:10,
    12:12
}
train['월(중식)'] = train['월'].map(month1)
test['월(중식)'] = test['월'].map(month1)

train['월(석식)'] = train['월'].map(month2)
test['월(석식)'] = test['월'].map(month2)

train['월(중식)'] = train['월(중식)'].astype('int') # float형이라 int로 변경
test['월(중식)'] = test['월(중식)'].astype('int')
train['월(석식)'] = train['월(석식)'].astype('int')
test['월(석식)'] = test['월(석식)'].astype('int')

In [40]:
train['공휴일전후'] = 0
test['공휴일전후'] = 0

In [41]:
train['공휴일전후'][4] = 1 #2
train['공휴일전후'][17] = 1 #
#train['공휴일전후'][62] = 1
# train['공휴일전후'][67] = 1
# train['공휴일전후'][82] = 1
train['공휴일전후'][131] = 1
# train['공휴일전후'][130] = 1
train['공휴일전후'][152] = 1
train['공휴일전후'][226] = 1
train['공휴일전후'][221] = 1
#train['공휴일전후'][222] = 1
train['공휴일전후'][224] = 1
train['공휴일전후'][225] = 1
# train['공휴일전후'][244] = 1
train['공휴일전후'][245] = 1
# train['공휴일전후'][267] = 1
train['공휴일전후'][310] = 2
train['공휴일전후'][311] = 1
train['공휴일전후'][309] = 1
train['공휴일전후'][330] = 1
train['공휴일전후'][379] = 1
train['공휴일전후'][467] = 1
# train['공휴일전후'][469] = 1
train['공휴일전후'][470] = 1
train['공휴일전후'][502] = 2
# train['공휴일전후'][501] = 1
train['공휴일전후'][511] = 1
train['공휴일전후'][565] = 1
train['공휴일전후'][623] = 1
train['공휴일전후'][651] = 1
# train['공휴일전후'][650] = 1
train['공휴일전후'][705] = 2
# train['공휴일전후'][707] = 1
train['공휴일전후'][709] = 2
# train['공휴일전후'][733] = 1
# train['공휴일전후'][748] = 1
# train['공휴일전후'][792] = 1
train['공휴일전후'][815] = 2
train['공휴일전후'][864] = 1
# train['공휴일전후'][863] = 1
train['공휴일전후'][950] = 1
train['공휴일전후'][951] = 1
train['공휴일전후'][953] = 2
train['공휴일전후'][954] = 1
train['공휴일전후'][955] = 1
train['공휴일전후'][971] = 1
# train['공휴일전후'][970] = 1
# train['공휴일전후'][1037] = 1
train['공휴일전후'][1038] = 1
train['공휴일전후'][1099] = 1
train['공휴일전후'][1129] = 2
# train['공휴일전후'][1128] = 1
train['공휴일전후'][1187] = 1
# train['공휴일전후'][1186] = 1

test['공휴일전후'][10] =2
test['공휴일전후'][20] = 1

In [42]:
# 원핫인코딩
train = pd.get_dummies(train, columns=['공휴일전후'])
test = pd.get_dummies(test, columns=['공휴일전후'])


In [43]:
test['공휴일전후_0'][20] = 1
test['공휴일전후_1'][20] = 0

# 코로나전후 추가 

In [44]:
train['코로나전후'] = 0
test['코로나전후'] = 0

In [45]:
train.loc[(train.년>=2020), '코로나전후'] = 1
test.loc[(test.년>=2020), '코로나전후'] = 1

In [46]:
# 원핫인코딩
test.코로나전후[0] = 0

train = pd.get_dummies(train, columns=['코로나전후'])
test = pd.get_dummies(test, columns=['코로나전후'])

test.코로나전후_0[0] = 0
test.코로나전후_1[0] = 1

In [47]:
train[train.석식계>0]['석식계'].mean() # 0이아닌값 평균

# 현재 1월만 대체
train['석식계'][244] = 478.8605851979346

#train['석식계'][281] = 398
train['석식계'][492] = 478.8605851979346

#train['석식계'][502] = 398
#train['석식계'][510] = 398
#train['석식계'][529] = 398
train['석식계'][730] = 478.8605851979346

#train['석식계'][747] = 398
#train['석식계'][766] = 398
train['석식계'][973] = 478.8605851979346

#train['석식계'][993] = 398

# 1 2 3 월 자기개발의날 대체.

### 최종제출 ( 자기개발1대체,금)

In [48]:
train.loc[(train.요일==4), '석식계'] -= 10

In [49]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor()

In [54]:
# train = train[train.석식계!= 0]

x1_train = train[['요일', '본사시간외근무명령서승인건수', '식사가능자수','월(중식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수','코로나전후_0','코로나전후_1']] #중식계
x2_train = train[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수','월(석식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']] #석식계

y1_train = train['중식계']
y2_train = train['석식계']

x1_test = test[['요일', '본사시간외근무명령서승인건수', '식사가능자수', '월(중식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수','코로나전후_0','코로나전후_1']]
x2_test = test[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수', '월(석식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']]

In [55]:
model1 = cat
model2 = cat

model1.fit(x1_train, y1_train) #중식계
pred1 = model1.predict(x1_test)

model2.fit(x2_train, y2_train) #석식계
pred2 = model2.predict(x2_test)

submission['중식계'] = pred1
submission['석식계'] = pred2

Learning rate set to 0.040347
0:	learn: 204.0286264	total: 4.02ms	remaining: 4.02s
1:	learn: 198.9277774	total: 6.65ms	remaining: 3.32s
2:	learn: 194.1772682	total: 12.3ms	remaining: 4.08s
3:	learn: 189.5957461	total: 17ms	remaining: 4.24s
4:	learn: 185.1673836	total: 19.7ms	remaining: 3.91s
5:	learn: 180.8821544	total: 22.6ms	remaining: 3.74s
6:	learn: 176.9285504	total: 31.8ms	remaining: 4.52s
7:	learn: 173.0449027	total: 39.6ms	remaining: 4.91s
8:	learn: 169.4941818	total: 43.8ms	remaining: 4.82s
9:	learn: 166.4415299	total: 48.7ms	remaining: 4.82s
10:	learn: 163.0336161	total: 52.3ms	remaining: 4.7s
11:	learn: 159.8610788	total: 55.6ms	remaining: 4.57s
12:	learn: 156.9432769	total: 58.2ms	remaining: 4.42s
13:	learn: 154.2465870	total: 61.2ms	remaining: 4.31s
14:	learn: 151.5503631	total: 64.7ms	remaining: 4.25s
15:	learn: 149.2578392	total: 69ms	remaining: 4.24s
16:	learn: 146.8414489	total: 72ms	remaining: 4.16s
17:	learn: 144.7537105	total: 74.6ms	remaining: 4.07s
18:	learn: 142.

In [56]:
submission.head(3)

,일자,중식계,석식계
0,2021-01-27,994.395609,321.318684
1,2021-01-28,924.868077,507.473362
2,2021-01-29,560.362565,233.021222


In [57]:
submission

,일자,중식계,석식계
0,2021-01-27,994.395609,321.318684
1,2021-01-28,924.868077,507.473362
2,2021-01-29,560.362565,233.021222
3,2021-02-01,1221.167374,518.737171
4,2021-02-02,978.508183,569.793711
5,2021-02-03,987.207832,515.775621
6,2021-02-04,917.826505,565.792804
7,2021-02-05,681.409333,369.800979
8,2021-02-08,1195.559017,683.592530
9,2021-02-09,1024.063567,585.701251


In [32]:
submission.mean()

중식계    948.562981
석식계    477.812709
dtype: float64

In [38]:
submission['석식계'].mean()

524.463584091015

In [58]:
submission.to_csv('자기개발1대체,금다운중식월랭크코로나.csv', index=False)

In [32]:
train.query('월<4 & 월>1').mean()['중식계']

970.355

In [22]:
submission.mean()['중식계']

988.7756025912

In [46]:
train.groupby('월').mean()['중식계'].sort_values()

월
12    827.336634
11    847.333333
7     848.736364
8     854.785047
6     863.311111
5     869.787879
4     883.621359
9     907.148936
10    917.771739
1     934.247525
3     953.607477
2     989.623656
Name: 중식계, dtype: float64

In [45]:
train.query('년>=2020').groupby('월').mean()['중식계'].sort_values()

월
6      760.000000
5      815.578947
8      825.100000
7      827.826087
10     852.684211
9      871.650000
11     878.095238
1      912.729730
4      924.650000
2      946.200000
3      958.090909
12    1048.500000
Name: 중식계, dtype: float64

In [47]:
train.query('년==2020').groupby('월').mean()['중식계'].sort_values()

월
6      760.000000
5      815.578947
8      825.100000
7      827.826087
1      830.300000
10     852.684211
9      871.650000
11     878.095238
4      924.650000
2      946.200000
3      958.090909
12    1048.500000
Name: 중식계, dtype: float64

In [50]:
train.query('년==2021& 월 == 1')['중식계'].mean()


1009.7058823529412

In [30]:
submission.mean()

중식계    948.562981
석식계    477.812709
dtype: float64

In [31]:
train.mean()

요일                   2.006639
본사정원수             2807.815768
본사휴가자수             157.913693
본사출장자수             241.142739
본사시간외근무명령서승인건수     274.117012
현본사소속재택근무자수         43.506224
중식계                890.334440
석식계                461.353894
요일(석식)               2.999170
년                 2018.052282
월                    6.512033
일                   15.947718
주                   26.570954
식사가능자수            2606.395851
월(중식)                6.469710
월(석식)                6.576763
공휴일전후_0              0.970954
공휴일전후_1              0.023237
공휴일전후_2              0.005809
코로나전후_0              0.793361
코로나전후_1              0.206639
dtype: float64

In [ ]:
train.q